In [1]:
import numpy as np
import pandas as pd
#import gym
import random
#import imageio
from tqdm.notebook import trange

import torch
from torch import nn
import torch.optim as optim

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

True
cuda


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
root_path = os.path.abspath('./drive/MyDrive/PAM Detection/')

Mounted at /content/drive


In [3]:
#user1 = pd.read_csv(os.path.join(root_path, 'user1_feat_act_time.csv'), index_col=0)
user2 = pd.read_csv(os.path.join(root_path, 'user2_feat_act_time.csv'), index_col=0)

#user2['seq_num'] = user2['seq_num'] + user1[-1:]['seq_num'].values + 1

#user1['PAM_Val'] = user1['PAM_Val'] - 1.0
user2['PAM_Val'] = user2['PAM_Val'] - 1.0

#user = pd.concat([user1, user2], axis=0)
user = user2

print(user)
print(user.shape)
#print(user1.shape)
#print(user2.shape)

                           Event_Type     X    Y  delta_time  seq_num  \
Time                                                                    
2021-09-07 08:40:02.135293       Move   917  654    0.000000        0   
2021-09-07 08:40:02.180209       Move   915  654    0.044916        0   
2021-09-07 08:40:02.247588       Move   912  654    0.067379        0   
2021-09-07 08:40:02.997540       Move   914  654    0.749952        0   
2021-09-07 08:40:03.003413       Move   914  652    0.005873        0   
...                               ...   ...  ...         ...      ...   
2021-09-14 16:17:10.821675       Move  1232  399    0.006835    11293   
2021-09-14 16:17:10.833393       Move  1232  402    0.011718    11293   
2021-09-14 16:17:10.840228       Move  1232  404    0.006835    11293   
2021-09-14 16:17:10.848042       Move  1232  407    0.007814    11293   
2021-09-14 16:17:10.856840       Move  1232  409    0.008798    11293   

                            PAM_Val  delta_x  delt

In [4]:
user = user[~user['seq_num'].duplicated(keep='last')]
user = user[user.columns.drop(list(user.filter(regex='delta')))]
user = user.fillna(0.0)


#user = user.filter(items=['PAM_Val', 'X', 'Y', 'elapsed_time', 'vel_mean', 'vel_min', 'vel_max'])
user = user.drop(columns=['seq_num', 'Event_Type'])
print(user)

                               X     Y  PAM_Val  x_vel_mean    x_vel_min  \
Time                                                                       
2021-09-07 08:40:04.818712   897   189      0.0  -52.590616 -1536.098310   
2021-09-07 08:40:17.166569   752   354      0.0  -34.032085 -3128.258603   
2021-09-07 08:40:19.432034  1005   304      0.0  769.946172  -513.083633   
2021-09-07 08:40:20.755192   965   306      0.0 -197.236583  -614.250614   
2021-09-07 08:40:43.144400   732   651      0.0  -72.861070 -5112.474438   
...                          ...   ...      ...         ...          ...   
2021-09-14 16:16:52.198967   310  1061      3.0 -346.354244 -5107.252298   
2021-09-14 16:16:58.590175   352   999      3.0   54.615781  -341.005968   
2021-09-14 16:17:00.818532   365   999      3.0   11.760806     0.000000   
2021-09-14 16:17:03.907202   673  1060      3.0  957.132526 -3073.770492   
2021-09-14 16:17:10.856840  1232   409      3.0  334.554710 -8196.721311   

           

In [5]:
minX = user['X'].min()
maxX = user['X'].max()
minY = user['Y'].min()
maxY = user['Y'].max()

print(minX, maxX, minY, maxY)

-5 1926 -3 1082


In [6]:
# bin data
bin_count = 50      # temporary until we find a better method for all features

def bin_data(user, bin_count, features):
  for feature in features:
    min = user[feature].min()
    max = user[feature].max()
    old_range = max - min

    #bin_start_loc = [x for x in range(min, max, (max - min) / bin_count)]
    #feature_binned = pd.cut(user[feature], bins=bin_start_loc, labels=range(bin_count-1))
    #user[feature] = feature_binned['binned']
    if old_range != 0:
      #print(user[(((user[feature] - min) / old_range) * (bin_count - 1)).isnull()])
      user[feature] = (((user[feature] - min) / old_range) * (bin_count - 1)).astype('int')

  return user

feature_cols = user.columns.to_list()
feature_cols.remove('PAM_Val')
user = bin_data(user, bin_count, feature_cols)

print(user)

                             X   Y  PAM_Val  x_vel_mean  x_vel_min  x_vel_max  \
Time                                                                            
2021-09-07 08:40:04.818712  22   8      0.0          30         48          0   
2021-09-07 08:40:17.166569  19  16      0.0          30         48          0   
2021-09-07 08:40:19.432034  25  13      0.0          31         48          1   
2021-09-07 08:40:20.755192  24  13      0.0          30         48          0   
2021-09-07 08:40:43.144400  18  29      0.0          30         47          0   
...                         ..  ..      ...         ...        ...        ...   
2021-09-14 16:16:52.198967   7  48      3.0          30         47          0   
2021-09-14 16:16:58.590175   9  45      3.0          30         48          0   
2021-09-14 16:17:00.818532   9  45      3.0          30         49          0   
2021-09-14 16:17:03.907202  17  48      3.0          32         48          0   
2021-09-14 16:17:10.856840  

In [7]:
#pam_groups = []
#for pam_num in range(0, 4):
#  pam_groups.append(user[user['PAM_Val'] == pam_num].groupby('seq_num'))

#print(len(pam_groups))

In [8]:
# Training parameters
n_training_episodes = 2
learning_rate = 0.7

# Evaluation parameters
n_eval_episodes = 10000

# Environment parameters
max_steps = 99
gamma = 0.95
eval_seed = []
map_size = [8, 8]

# Exploration parameters
max_epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.0  #0.0005

In [9]:
#class environment:
# '''
#  Parameters:
#    minX, maxX, minY, maxY: all inclusive
#  '''
#
#
#  class obervation_space_class:
#    def __init__(self, minX, maxX, minY, maxY):
#      self.minX = minX
#      self.minY = minY
#     self.maxX = maxX
#     self.maxY = maxY
#      self.n = [maxX-minX+1, maxY-minY+1]
#
#    def sample(self):
#      return np.array([random.randint(self.minX, self.maxX), random.randint(self.minY, self.maxY)])
#
#  class action_space_class:
#    def __init__(self):
#      self.actions_to_transition = np.array([
#         [0, 1],
#         [1, 1],
#         [1, 0],
#         [1, -1],
#         [0, -1],
#         [-1, -1],
#         [-1, 0],
#         [-1, 1]
#      ])
#      self.actions = range(len(self.actions_to_transition))
#      self.n = len(self.actions)
#
#    def sample(self):
#      return random.randint(0, self.n - 1)
#
#    def take_action(self, action):
#      return np.array(self.actions_to_transition[action])
#
#
#  def __init__(self, minX, maxX, minY, maxY):
#    self.minX = minX
#    self.minY = minY
#    self.maxX = maxX
#    self.maxY = maxY
#    self.observation_space = self.obervation_space_class(self.minX, self.maxX, self.minY, self.maxY)
#    self.action_space = self.action_space_class()
#    self.goal = np.array([0, 0])
#    print('goal:', self.goal)
#
#    self.location = self.observation_space.sample()
#    while(np.array_equal(self.location, self.goal)):
#      self.location = self.observation_space.sample()
#
#  def step(self, action):
#    old_location = self.location
#    new_location = self.location + self.action_space.take_action(action)
#    if np.array_equal(new_location, self.goal):
#      is_done = True
#    else:
#      is_done = False
#
#    if new_location[0] < self.minX or new_location[0] > self.maxX or new_location[1] < self.minY or new_location[1] > self.maxY:
#      location = old_location
#      #reward = -1
#    else:
#      location = new_location
#      self.location = new_location
#
#    reward = np.sqrt(np.sum(np.square(old_location - self.goal))) - np.sqrt(np.sum(np.square(location - self.goal)))
#
#    #out_location = location[0] * self.maxX + location[1]
#    out_location = location
#
#    return out_location, reward, is_done, False, {'prob': 1}
#
#  def reset(self):
#    self.location = self.observation_space.sample()
#    while(np.array_equal(self.location, self.goal)):
#      self.location = self.observation_space.sample()
#    #out_location = self.location[0] * self.maxX + self.location[1]
#    out_location = self.location
#    return out_location
#
#env = environment(minX, maxX, minY, maxY)

In [10]:
#print("Observation Space", env.observation_space.n)
#print("Sample observation", env.observation_space.sample()) # display a random observation
#print("Action Space Shape", env.action_space.n)
#print("Action Space Sample", env.action_space.sample())

In [11]:
#state_space = env.observation_space.n
#action_space = env.action_space.n

#print("There are ", state_space, " possible states")
#print("There are ", action_space, " possible actions")

In [12]:
#def initialize_q_table(state_space, action_space_list):
#  '''
#  action_space_list: list of action spaces for each feature
#
#  returns qtable with layout of [feature (list), state (numpy), value]
#  '''
#  Qtable = []
#  for action_space in action_space_list:
#    Qtable.append(np.zeros((state_space, action_space)))
#  return Qtable
#
# [feature (list), state (numpy), value]
#Qtable = initialize_q_table(len(user), [bin_count for x in range(len(feature_cols))])

In [13]:
# "acting policy": how the agent acts when trying to learn
#def copy_demonstration(demonstration_timestamp):
#  return demonstration_timestamp['Action']

#def epsilon_greedy_policy(Qtable, state, epsilon):
#  action = []
#  random_int = random.uniform(0,1)
#  if random_int > epsilon:  # follow Q-table
#    for feature_qtable in Qtable:
#      action.append(np.argmax(feature_qtable[state]))
#  else:                     # randomly explore
#    for feature_qtable in Qtable:
#      action.append(feature_qtable[random.randint(0,len(feature_qtable)-1)])
#  return action

# "policy/final policy": the policy to use when agent is finished learning
#def greedy_policy(Qtable, state):
#  action = []
#  for feature_qtable in Qtable:
#    action.append(np.argmax(feature_qtable[state]))
#  return action


class NN_policy(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(NN_policy, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    #self.rnn = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
    self.fc1 = nn.Linear(self.input_size, self.hidden_size)
    self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
    self.fc3 = nn.Linear(self.hidden_size, self.output_size)
    self.relu = nn.ReLU()
    #self.sm = nn.Softmax(dim=0)    # not needed if using Cross Entropy loss

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    out = self.fc3(x)

    return out

policy_dict = {}
for feature in feature_cols:
  policy = NN_policy(len(feature_cols)+1, 8, bin_count)
  policy = policy.to(device)
  policy_dict.update({feature: policy})

In [14]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, max_steps, policy_dict, demonstrations):
  def epsilon_policy_check(epsilon):
    random_int = random.uniform(0,1)
    if random_int > epsilon:  # follow Q values
      return True
    else:                     # randomly explore
      return False

  def determine_reward(pred, actual):
    return 1 / abs(pred - actual)

  loss_fn = nn.MSELoss()


  optimizer_dict = {}
  for feature, policy in policy_dict.items():
    policy.train()
    optimizer_dict.update({feature: optim.Adam(policy.parameters(), lr=0.0001)})

  for episode in trange(n_training_episodes):

    # decay epsilon after every episode
    # aka explore less
    #epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    epsilon = max_epsilon

    # Reset the environment
    #state = env.reset()
    #done = False

    # repeat
    for traj_num in trange(len(demonstrations)-1):
      #for timestep in range(len(trajectory)):
      # pick action and move to new state
      #state = demonstrations.iloc[traj_num].drop('PAM_Val')
      state = demonstrations.iloc[traj_num]
      state = torch.Tensor([state]).to(device)

      for (feature, policy), (feature, optimizer) in zip(policy_dict.items(), optimizer_dict.items()):
        prediction = policy(state)
        if epsilon_policy_check(epsilon):
          action = torch.argmax(prediction)
        else:
          action = random.randint(0, bin_count-1)
        #print(action)
        #print(torch.reshape(prediction, (-1, 1))[action])
        prediction = torch.reshape(prediction, (-1, 1))[action]
        action = torch.Tensor([action]).type(torch.cuda.LongTensor).to(device)
        #print(action)
        #action = epsilon_greedy_policy(Qtable, state, epsilon)
        #new_state, reward, done, info, probability = env.step(action)

        reward = determine_reward(action, demonstrations.iloc[traj_num][feature])

        # update q-table
        #print(state, new_state)
        #Qtable[state[0]][state[1]][action] = Qtable[state[0]][state[1]][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state[0]][new_state[1]]) - Qtable[state[0]][state[1]][action])
        next_state = demonstrations.iloc[traj_num+1]
        next_state = torch.Tensor([next_state]).to(device)
        target = reward + gamma * torch.max(policy(next_state))

        #print('==================')
        #print(prediction)
        #print(target)
        loss = loss_fn(prediction, target)
        #print(loss)

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

        # If done, finish the episode
        #if done:
        #  break

        # Our state is the new state
        #state = new_state
  return policy_dict


policy_dict = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, max_steps, policy_dict, user)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/11293 [00:00<?, ?it/s]

  0%|          | 0/11293 [00:00<?, ?it/s]

In [15]:
def evaluate_agent(policy_dict, demonstrations):

  #episode_rewards = []
  #for episode in trange(n_eval_episodes):
  #  if seed:
  #    state = env.reset(seed=seed[episode])
  #  else:
  #    state = env.reset()
  #  done = False
  #  total_rewards_ep = 0
  #
  #  for step in range(max_steps):
  #    # Take the action (index) that have the maximum reward
  #    #action = np.argmax(Qtable[state][:])
  #    action = greedy_policy(Qtable, state)
  #    new_state, reward, done, info, probability = env.step(action)
  #    total_rewards_ep += reward
  #
  #    if done:
  #      break
  #    state = new_state
  #
  #  episode_rewards.append(total_rewards_ep)
  #
  #mean_reward = np.mean(episode_rewards)
  #std_reward = np.std(episode_rewards)
  #
  #return mean_reward, std_reward

  for feature, policy in policy_dict.items():
    policy.eval()

  pred = []
  actual = []
  with torch.no_grad():
    for traj_num in trange(len(demonstrations)-1):
      state = demonstrations.iloc[traj_num]
      state = torch.Tensor([state]).to(device)

      for feature, policy in policy_dict.items():
        action = torch.argmax(policy(state))

        pred.append(action.to('cpu').numpy())
        actual.append(demonstrations.iloc[traj_num]['PAM_Val'])


  conf_matrix = confusion_matrix(pred, actual, labels=[*range(0, bin_count)])
  accuracy = accuracy_score(pred, actual)
  precision = precision_score(pred, actual, labels=[*range(0, bin_count)], average='macro')
  recall = recall_score(pred, actual, labels=[*range(0, bin_count)], average='macro')
  f1 = f1_score(pred, actual, labels=[*range(0, bin_count)], average='macro')

  print('Confusion Matrix:')
  print(conf_matrix)
  print(conf_matrix.sum(axis=0))
  print('Accuracy:')
  print(accuracy)
  print('Precision:')
  print(precision)
  print('Recall:')
  print(recall)
  print('F1:')
  print(f1)


evaluate_agent(policy_dict, user)

  0%|          | 0/11293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix:
[[2328 6001 2497 ...    0    0    0]
 [   4   18    4 ...    0    0    0]
 [ 111  279  113 ...    0    0    0]
 ...
 [ 657 1454  662 ...    0    0    0]
 [2637 6711 2836 ...    0    0    0]
 [ 581 1372  608 ...    0    0    0]]
[ 82843 214267  88800  31931      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]
Accuracy:
0.008072448610835222
Precision:
0.0011616419833847733
Recall:
0.023639755308989256
F1:
0.0018815857598368268
